<a href="https://colab.research.google.com/github/anthonyendara123/BASE_LABS_TEST/blob/main/BIBITOR_BASE_LABS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Start of the setup and installation process for the Spark environment with SQLite support

This section performs the necessary installation and configuration to run Apache Spark in this working environment (Google Colab or similar), including:

Installation of OpenJDK 8, required to run Spark.

Extraction of the Spark binary package (version 3.2.3 with Hadoop 3.2).

Download of the JDBC driver for SQLite (sqlite-jdbc-3.34.0.jar), necessary for Spark to read and write SQLite tables via JDBC.

Configuration of the environment variables JAVA_HOME and SPARK_HOME so that Spark and Java work correctly.

Initialization of findspark to integrate Spark with the Python environment.

Creation of the Spark session with the configuration to load the SQLite JDBC driver.

This configuration is essential to be able to work with Spark and SQLite together, since the goal of the technical test is to demonstrate skills in integrating data from SQLite databases in a Spark environment, processing it, and analyzing it efficiently.



In [ ]:
!wget -q https://archive.apache.org/dist/spark/spark-3.2.3/spark-3.2.3-bin-hadoop3.2.tgz

In [ ]:
# install JAVA 8 SDK
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
# extract the Spark version
!tar xf spark-3.2.3-bin-hadoop3.2.tgz
!wget https://repo1.maven.org/maven2/org/xerial/sqlite-jdbc/3.34.0/sqlite-jdbc-3.34.0.jar

--2025-07-08 21:33:07--  https://repo1.maven.org/maven2/org/xerial/sqlite-jdbc/3.34.0/sqlite-jdbc-3.34.0.jar
Resolving repo1.maven.org (repo1.maven.org)... 199.232.192.209, 199.232.196.209, 2a04:4e42:4c::209, ...
Connecting to repo1.maven.org (repo1.maven.org)|199.232.192.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7296329 (7.0M) [application/java-archive]
Saving to: ‘sqlite-jdbc-3.34.0.jar.2’

sqlite-jdbc-3.34.0. 100%[===================>]   6.96M  --.-KB/s    in 0.09s   

2025-07-08 21:33:07 (81.7 MB/s) - ‘sqlite-jdbc-3.34.0.jar.2’ saved [7296329/7296329]



In [ ]:
# set environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.3-bin-hadoop3.2"

# download findspark library
!pip install -q findspark
!pip install -q pyspark

In [ ]:
import findspark

findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Leer SQLite") \
    .config("spark.jars", "sqlite-jdbc-3.34.0.jar") \
    .getOrCreate()

# The configuration and installation of Spark and the SQLite environment has completed successfully.
Now, the environment is ready, and we begin the main process of loading and processing data.

In [ ]:
import sqlite3
import gdown
import zipfile
import os
import re
import pandas as pd
import time, datetime
from pyspark.sql import functions as F
from pyspark.sql.functions import format_number
import shutil
from glob import glob

ENTITY = "BIBITOR"
base_folder = "/content/final_reports"
db_path = "/content/mi_base.db"

###################################################
#                                                 #
# == 1. DEFINICIÓN DE FUNCIONES PERSONALIZADAS == #
#                                                 #
###################################################
def extract_id(google_drive_url):
    """
    Extracts the unique ID from a Google Drive link.

    This function takes a Google Drive URL and searches for a specific pattern
    that identifies the file ID (the part following '/d/' in the URL).
    If the ID is found, it returns it as a string; otherwise, it returns None.
    """
    match = re.search(r'/d/([a-zA-Z0-9_-]+)', google_drive_url)
    return match.group(1) if match else None

def download_files(wget1, wget2, wget3, wget4):
    """
    Executes a series of download commands using wget.

    It takes four commands (wget) as arguments and executes them one by one.
    For each command:
    - Prints the number of the download being executed.
    - Executes the command and displays the output line by line.
    - Captures and displays any errors that occur during the download.
    At the end, it indicates that all downloads were completed.
    """
    comands = [wget1, wget2, wget3, wget4]
    for i, cmd in enumerate(comands, 1):
        try:
            print(f"\n Running download {i}...")
            salida = get_ipython().getoutput(cmd)
            for linea in salida:
                print(linea)
        except Exception as e:
            print(f" Download error {i}: {e}")
    print("✅ Downloads completed.\n")

def download_and_extract_zip(url, nombre_zip="archivo.zip", destination_folder="archivos"):
    """
    Downloads a ZIP file from a Google Drive link and unzips it.

    Steps:
    1. Extracts the file ID from the Google Drive URL.
    2. Downloads the ZIP file using 'gdown' and saves it with the specified name.
    3. Creates the destination folder if it doesn't exist.
    4. Unzips the ZIP file into the destination folder.
    5. Lists the extracted files in the console.

    Parameters:
    - url: Google Drive link to the ZIP file.
    - zip_name: Local name to save the ZIP file (default "file.zip").
    - destination_folder: Folder where the files are extracted (default "files").

    Handles exceptions to display errors during download or extraction.
    """
    try:
        file_id = extract_id(url)
        if not file_id:
            print(f" Could not extract link ID: {url}")
            return

        print(f"\n Downloading ZIP from: {url}")
        gdown.download(f"https://drive.google.com/uc?id={file_id}", nombre_zip, quiet=False)

        print(f" Unzipping into: {destination_folder}/")
        os.makedirs(destination_folder, exist_ok=True)
        with zipfile.ZipFile(nombre_zip, 'r') as zip_ref:
            zip_ref.extractall(destination_folder)

        print(f" Extracted content: ")
        for f in os.listdir(destination_folder):
            print("  -", f)

    except Exception as e:
        print(f" Error downloading or unzipping the ZIP: {e}")

def load_tables_from_result(results_tbl, db_path):
    """
    Loads tables from an SQLite database into Spark and registers them as temporary views.

    Parameters:
    - results_tbl: List of tuples with table names, e.g. [('table1',), ('table2',), ...]
    - db_path: Path to the SQLite database file.

    Process:
    1. Extracts the table names from the list of tuples.
    2. Loads each table into a Spark DataFrame using JDBC.
    3. Registers each DataFrame as a temporary view with the table name.
    4. Saves the DataFrames in a dictionary with the table as the key.

    Returns:
    - A dictionary where the key is the table name and the value is the loaded DataFrame.
    """
    dfs = {}

    # Extract table names from [('table1',), ('table2',)...]
    table_names = [t[0] for t in results_tbl]

    for name in table_names:
        df = spark.read.format("jdbc") \
            .option("url", f"jdbc:sqlite:{db_path}") \
            .option("dbtable", name) \
            .option("driver", "org.sqlite.JDBC") \
            .load()

        df.createOrReplaceTempView(name)
        dfs[name] = df
        print(f" Table '{name}' loaded and registered.")

    return dfs

def save_csv(df, name):
    """
    Saves a PySpark DataFrame as a single CSV file within a defined base folder.

    Parameters:
    -----------
    df : pyspark.sql.DataFrame
        DataFrame to be saved in CSV format.
    name : str
        Base name for the temporary folder and the final CSV file.

    Process:
    --------
    1. Builds a temporary path by combining the base folder and the provided name.
    2. Writes the DataFrame in CSV format to this temporary path, consolidating the data into a single file
       using coalesce(1). Any previous content is overwritten.
    3. Searches for the generated CSV file within the temporary folder (with prefix 'part-').
    4. Moves this CSV file to the final path named "{name}.csv" within the base folder.
    5. Deletes the temporary folder used for writing, leaving only the final CSV file.

    This ensures the result is a single, properly named CSV file, avoiding partitioned file fragments and
    keeping the storage structure organized.
    """
    temp_path = os.path.join(base_folder, name)
    df.coalesce(1).write.csv(temp_path, header=True, mode="overwrite")
    csv_file = glob(os.path.join(temp_path, "part-*.csv"))[0]
    final_path = os.path.join(base_folder, f"{name}.csv")
    shutil.move(csv_file, final_path)
    shutil.rmtree(temp_path)

################################################
#                                              #
# ==     2. process setting block           == #
#                                              #
################################################
start_time = time.time()
print(f"Start of process for ENTITY {ENTITY}: {datetime.datetime.fromtimestamp(start_time)}")

try:
    ENV_VALUE = 1
    table = "params" if ENV_VALUE == 1 else "params_des"
    print(f"== Selected environment is: {ENV_VALUE} ==")
    print(f"== Using table: {table} ==")

    print("== Establishing connection with SQLite database and creating a cursor to execute queries ==")
    conn = sqlite3.connect('mi_base.db')
    cursor = conn.cursor()

    print("== Creating the specified table if it does not exist, with columns ENTITY, PARAMETRO, and VALOR ==")
    cursor.execute(f'''
    CREATE TABLE IF NOT EXISTS {table} (
        ENTITY TEXT,
        PARAMETRO TEXT,
        VALOR TEXT
    )''')

    print(f"== Deleting all rows from the table where the ENTITY column equals the variable {ENTITY}, and committing changes ==")
    cursor.execute(f"DELETE FROM {table} WHERE ENTITY = ?", (ENTITY,))
    conn.commit()

    print("== Inserting list of tuples with pairs (variable name, value) into the inserts variable ==")
    print("== Variables starting with VAR_GET_ contain commands or URLs to download CSV files ==")
    print("== Variables starting with VAR_tbl_ contain the names of tables where those data will be stored ==")
    inserts = [
        ("VAR_GET_2017PurchasePricesDec", 'wget --no-check-certificate "https://drive.google.com/uc?export=download&id=1dGNwXDhVuVzvHAz0yWhE99luxT1B1G0d" -O 2017PurchasePricesDec.csv'),
        ("VAR_GET_BegInvFINAL12312016", 'wget --no-check-certificate "https://drive.google.com/uc?export=download&id=1h5t-HUJvryZiPMRzIdtYXdh4-ofmMCVK" -O BegInvFINAL12312016.csv'),
        ("VAR_GET_EndInvFINAL12312016", 'wget --no-check-certificate "https://drive.google.com/uc?export=download&id=1hzZhhfWr6sIO_lxIN9IkSlARjjNjE6pB" -O EndInvFINAL12312016.csv'),
        ("VAR_GET_InvoicePurchases12312016", 'wget --no-check-certificate "https://drive.google.com/uc?export=download&id=10zwe070P7gCSQ7cDBkW5nOHTt45o7ya7" -O InvoicePurchases12312016.csv'),
        ("VAR_GET_PurchasesFINAL12312016", 'https://drive.google.com/file/d/1W1VIx6grLFjoqOWvyg66BBGt06Q7X7Xd/view?usp=drive_link'),
        ("VAR_GET_SalesFINAL12312016", 'https://drive.google.com/file/d/1APCHTFo79T0bd_ZANbjdwJkS8uV8xDHY/view?usp=sharing'),
        ("VAR_tbl_2017PurchasePricesDec", 'PricingPurchasesDec'),
        ("VAR_tbl_BegInvFINAL12312016", 'BegInvDec'),
        ("VAR_tbl_EndInvFINAL12312016", 'EndInvDec'),
        ("VAR_tbl_InvoicePurchases12312016", 'VendorInvoicesDec'),
        ("VAR_tbl_PurchasesFINAL12312016", 'PurchasesDec'),
        ("VAR_tbl_SalesFINAL12312016", 'SalesDec'),
        ("ETAPA", "1"),
    ]

    print(f"== Inserting parameters and values into SQLite table for specified ENTITY: {ENTITY} and committing changes ==")
    for param, value in inserts:
        print(f"Inserting -> ENTITY: {ENTITY}, PARAMETRO: {param}, VALOR: {value}")
        cursor.execute(f"INSERT INTO {table} (ENTITY, PARAMETRO, VALOR) VALUES (?, ?, ?)", (ENTITY, param, value))
    conn.commit()

    print(f"== Getting the value of ETAPA for ENTITY: {ENTITY} and converting to integer ==")
    cursor.execute(f"SELECT VALOR FROM {table} WHERE ENTITY = ? AND PARAMETRO = 'ETAPA'", (ENTITY,))
    row = cursor.fetchone()
    if row:
        stage = int(row[0])  # convert to integer
        print(f"🔹 ETAPA found: {stage} (type: {type(stage).__name__})")
    else:
        raise ValueError("ETAPA value not found for ENTITY.")

    te_time = time.time()
    print(f"End of block 2 (PROCESS ENVIRONMENT SETUP) ", datetime.datetime.fromtimestamp(te_time))
    elapsed_time = te_time - start_time
    print(f"Block 2 finished in : {elapsed_time:.2f} seconds")

    ################################################
    #                                              #
    # ==    3. DATA SOURCES LOAD BLOCK          == #
    #                                              #
    ################################################

    if stage == 1:
        ts_time = time.time()
        print(f"Start of block 3 (LOAD CSV DATA SOURCES) STAGE {stage}", datetime.datetime.fromtimestamp(ts_time))
        print(f"== Query and extract 6 VAR_GET_ values for ENTITY: {ENTITY} ==")
        sql = f"SELECT VALOR FROM {table} WHERE ENTITY = ? AND PARAMETRO LIKE 'VAR_GET_%'"
        cursor.execute(sql, (ENTITY,))
        results = cursor.fetchall()

        if len(results) == 6:
            val_1, val_2, val_3, val_4, val_5, val_6 = [row[0] for row in results]
        else:
            raise ValueError(f"Expected 6 values but found {len(results)}.")

        print("== Download files using the first 4 obtained URLs/commands ==")
        download_files(val_1, val_2, val_3, val_4)

        print("== Download and extract ZIPs from the specified URLs ==")
        download_and_extract_zip(val_6)
        download_and_extract_zip(val_5)

        print("== Read CSV files into pandas DataFrames ==")
        df1 = pd.read_csv("/content/2017PurchasePricesDec.csv")
        df2 = pd.read_csv("archivos/SalesFINAL12312016.csv")
        df3 = pd.read_csv("/content/BegInvFINAL12312016.csv")
        df4 = pd.read_csv("/content/EndInvFINAL12312016.csv")
        df5 = pd.read_csv("/content/InvoicePurchases12312016.csv")
        df6 = pd.read_csv("/content/archivos/PurchasesFINAL12312016.csv")

        print("== Info df1 (2017PurchasePricesDec) ==")
        df1.info()
        print("\n== Info df2 (SalesFINAL12312016) ==")
        df2.info()
        print("\n== Info df3 (BegInvFINAL12312016) ==")
        df3.info()
        print("\n== Info df4 (EndInvFINAL12312016) ==")
        df4.info()
        print("\n== Info df5 (InvoicePurchases12312016) ==")
        df5.info()
        print("\n== Info df6 (PurchasesFINAL12312016) ==")
        df6.info()

        print("\n== Extract VAR_tbl_ values and assign to variables ==")
        sql_tbl = f"SELECT VALOR FROM {table} WHERE ENTITY = ? AND PARAMETRO LIKE 'VAR_tbl_%'"
        cursor.execute(sql_tbl, (ENTITY,))
        results_tbl = cursor.fetchall()

        if len(results_tbl) == 6:
            tbl_1, tbl_2, tbl_3, tbl_4, tbl_5, tbl_6 = [row[0] for row in results_tbl]
        else:
            raise ValueError(f"Expected 6 VAR_tbl_ values but found {len(results_tbl)}.")

        print("== Create dictionary of table names and DataFrames ==")
        tables_df = {
            tbl_1: df1,
            tbl_2: df2,
            tbl_3: df3,
            tbl_4: df4,
            tbl_5: df5,
            tbl_6: df6
        }

        print("== Save DataFrames to SQLite if not empty ==")
        for table_name, df in tables_df.items():
            if not df.empty:
                print(f"Saving {table_name} ({len(df)} rows)...")
                df.to_sql(table_name, conn, if_exists="replace", index=False)
            else:
                print(f"{table_name} is empty. Skipping save.")

        print(f"Process {ENTITY} completed successfully")
        print(f"Updating stage for {ENTITY} to 2")
        stage += 1
        sql_update = f"""
        UPDATE {table}
        SET VALOR = ?
        WHERE ENTITY = ? AND PARAMETRO = 'ETAPA'
        """

        cursor.execute(sql_update, (str(stage), ENTITY))
        conn.commit()

        cursor.execute(f"SELECT * FROM {table}")
        rows = cursor.fetchall()

        for row in rows:
            print(row)

        te_time = time.time()
        print(f"End of block 3 (LOAD CSV DATA SOURCES) STAGE 1", datetime.datetime.fromtimestamp(te_time))
        elapsed_time_load = te_time - ts_time
        print(f"(LOAD CSV DATA SOURCES) STAGE 1 completed in:", datetime.datetime.fromtimestamp(elapsed_time_load))

    ##############################################
    #                                            #
    # ==    4. LOADING TABLES INTO DATAFRAMES  == #
    #                                            #
    ##############################################
    if stage == 2:
        ts_time = time.time()
        print(f"Start of block 4 (LOAD TABLES TO DATAFRAMES AND BUSINESS LOGIC) STAGE {stage}", datetime.datetime.fromtimestamp(ts_time))

        print("== Calling function load_tables_from_result to load tables into dfs and then save them into their corresponding dataframes ==")
        dfs = load_tables_from_result(results_tbl, db_path)
        df_SalesDec = dfs["SalesDec"]
        df_PricingPurchasesDec = dfs["PricingPurchasesDec"]
        df_BegInvDec = dfs["BegInvDec"]
        df_EndInvDec = dfs["EndInvDec"]
        df_VendorInvoicesDec = dfs["VendorInvoicesDec"]
        df_PurchasesDec = dfs["PurchasesDec"]

        print("== Performing printSchema() on dataframe df_SalesDec ==")
        df_SalesDec.printSchema()

        print("== Performing printSchema() on dataframe df_PricingPurchasesDec ==")
        df_PricingPurchasesDec.printSchema()

        print("== Performing printSchema() on dataframe df_BegInvDec and caching it because it's heavy and we need it in memory ==")
        df_BegInvDec.cache().printSchema()

        print("== Performing printSchema() on dataframe df_EndInvDec ==")
        df_EndInvDec.printSchema()

        print("== Performing printSchema() on dataframe df_VendorInvoicesDec ==")
        df_VendorInvoicesDec.printSchema()

        print("== Performing printSchema() on dataframe df_PurchasesDec ==")
        df_PurchasesDec.printSchema()

        #######################################
        #                                     #
        # ==    5. BUSINESS LOGIC          == #
        #                                     #
        #######################################
        print("== Selecting necessary columns from Pricing dataframe ==")
        df_cost = df_PricingPurchasesDec.select("Brand", "Description", "Size", "PurchasePrice")

        print("== Joining sales dataframe with cost info from pricing dataframe ==")
        df_sales_with_cost = df_BegInvDec.join(
            df_cost,
            on=["Brand", "Description", "Size"],
            how="left"
        )

        print("== Calculating Profit ($), Revenue ($), and Margin (%) ==")
        df_sales_profit = df_sales_with_cost.withColumn(
            "Profit",
            (F.col("SalesPrice") - F.col("PurchasePrice")) * F.col("SalesQuantity")
        ).withColumn(
            "Revenue",
            F.col("SalesPrice") * F.col("SalesQuantity")
        ).withColumn(
            "Margin",
            F.when(F.col("Revenue") != 0, (F.col("Profit") / F.col("Revenue")) * 100).otherwise(0)
        )

        print("== Getting Top 10 products by Profit ==")
        top10_products_profit = df_sales_profit.orderBy(F.desc("Profit")).limit(10)
        top10_products_profit.show()

        print("== Getting Top 10 products by Margin ==")
        top10_products_margin = df_sales_profit.orderBy(F.desc("Margin")).limit(10)
        top10_products_margin.show()

        print("== Aggregating Profit and Revenue by Brand ==")
        df_brands = df_sales_profit.groupBy("Brand").agg(
            F.sum("Profit").alias("TotalProfit"),
            F.sum("Revenue").alias("TotalRevenue")
        ).withColumn(
            "Margin",
            F.when(F.col("TotalRevenue") != 0, (F.col("TotalProfit") / F.col("TotalRevenue")) * 100).otherwise(0)
        )

        print("== Getting Top 10 brands by Profit ==")
        top10_brands_profit = df_brands.orderBy(F.desc("TotalProfit")).limit(10)
        top10_brands_profit.show()

        print("== Getting Top 10 brands by Margin ==")
        top10_brands_margin = df_brands.orderBy(F.desc("Margin")).limit(10)
        top10_brands_margin.show()

        print("== Filtering products losing money (Profit < 0) ==")
        products_losing_money = df_sales_profit.filter(F.col("Profit") < 0)
        products_losing_money.show()

        print("== Filtering brands losing money (TotalProfit < 0) ==")
        brands_losing_money = df_brands.filter(F.col("TotalProfit") < 0)
        brands_losing_money.show()

        print("== Saving reports using save_csv function ==")

        te_time = time.time()
        print(f"End of blocks 4, 5 and 6 (LOAD TABLES TO DATAFRAMES, BUSINESS LOGIC AND REPORT SAVING) STAGE 2 ", datetime.datetime.fromtimestamp(te_time))
        elapsed_time_carga = te_time - ts_time
        print(f"(LOAD TABLES TO DATAFRAMES, BUSINESS LOGIC AND REPORT SAVING) STAGE 2 completed in :", datetime.datetime.fromtimestamp(elapsed_time_carga))

        #################################################################
        #                                                               #
        # ==    6.Save resulting files from the queries               == #
        #                                                               #
        #################################################################
        print("== We call the save_csv function to save the dfs in our path /content/final_reports/....... to be able to download them and use as sources ==")
        save_csv(top10_products_profit, "Top10_Products_Profit")
        save_csv(top10_products_margin, "Top10_Products_Margin")

        save_csv(top10_brands_profit, "Top10_Brands_Profit")
        save_csv(top10_brands_margin, "Top10_Brands_Margin")

        save_csv(products_losing_money, "Products_Losing_Money")
        save_csv(brands_losing_money, "Brands_Losing_Money")

        print(f"End of blocks 4, 5 and 6 (LOAD TABLES TO DATAFRAMES, BUSINESS LOGIC AND REPORT SAVING) STAGE 2 ", datetime.datetime.fromtimestamp(te_time))
        elapsed_time_carga = te_time - ts_time
        print(f"(LOAD TABLES TO DATAFRAMES, BUSINESS LOGIC AND REPORT SAVING) STAGE 2 completed in :", datetime.datetime.fromtimestamp(elapsed_time_carga))

except Exception as e:
    print(f"\nGeneral process error: {e}")

finally:
    print("== Closing SQLite connection if open ==")
    if 'conn' in locals():
        conn.close()
        print("SQLite connection closed.")

    print("== Releasing memory by deleting used variables ==")
    vars_to_delete = ['df1', 'df2', 'df3', 'df4', 'df5', 'df6',
                      'valor_1', 'valor_2', 'valor_3', 'valor_4', 'valor_5', 'valor_6',
                      'tablas_df', 'cursor', 'tabla', 'ENTITY']
    for var in vars_to_delete:
        if var in locals():
            del locals()[var]

    end_time = time.time()
    elapsed_time = end_time - start_time
    print("End of process:", datetime.datetime.fromtimestamp(end_time))
    print(f"Process completed in {elapsed_time:.2f} seconds")
    print("== Closing spark session to free resources ==")

    #spark.stop()

Start of process for ENTITY BIBITOR: 2025-07-08 21:33:25.482165
== Selected environment is: 1 ==
== Using table: params ==
== Establishing connection with SQLite database and creating a cursor to execute queries ==
== Creating the specified table if it does not exist, with columns ENTITY, PARAMETRO, and VALOR ==
== Deleting all rows from the table where the ENTITY column equals the variable BIBITOR, and committing changes ==
== Inserting list of tuples with pairs (variable name, value) into the inserts variable ==
== Variables starting with VAR_GET_ contain commands or URLs to download CSV files ==
== Variables starting with VAR_tbl_ contain the names of tables where those data will be stored ==
== Inserting parameters and values into SQLite table for specified ENTITY: BIBITOR and committing changes ==
Inserting -> ENTITY: BIBITOR, PARAMETRO: VAR_GET_2017PurchasePricesDec, VALOR: wget --no-check-certificate "https://drive.google.com/uc?export=download&id=1dGNwXDhVuVzvHAz0yWhE99luxT1B1G

Downloading...
From (original): https://drive.google.com/uc?id=1APCHTFo79T0bd_ZANbjdwJkS8uV8xDHY
From (redirected): https://drive.google.com/uc?id=1APCHTFo79T0bd_ZANbjdwJkS8uV8xDHY&confirm=t&uuid=e1caf9ea-2fc7-4216-b5a1-e8ea35041738
To: /content/archivo.zip
100%|██████████| 150M/150M [00:00<00:00, 239MB/s]


 Unzipping into: archivos/
 Extracted content: 
  - SalesFINAL12312016.csv
  - PurchasesFINAL12312016.csv



Downloading...
From (original): https://drive.google.com/uc?id=1W1VIx6grLFjoqOWvyg66BBGt06Q7X7Xd
From (redirected): https://drive.google.com/uc?id=1W1VIx6grLFjoqOWvyg66BBGt06Q7X7Xd&confirm=t&uuid=06b0548e-e9b8-4d41-980d-d59318b23035
To: /content/archivo.zip
100%|██████████| 45.9M/45.9M [00:00<00:00, 170MB/s]


 Unzipping into: archivos/
 Extracted content: 
  - SalesFINAL12312016.csv
  - PurchasesFINAL12312016.csv
== Read CSV files into pandas DataFrames ==
== Info df1 (2017PurchasePricesDec) ==
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12261 entries, 0 to 12260
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Brand           12261 non-null  int64  
 1   Description     12260 non-null  object 
 2   Price           12261 non-null  float64
 3   Size            12260 non-null  object 
 4   Volume          12260 non-null  object 
 5   Classification  12261 non-null  int64  
 6   PurchasePrice   12261 non-null  float64
 7   VendorNumber    12261 non-null  int64  
 8   VendorName      12261 non-null  object 
dtypes: float64(2), int64(3), object(4)
memory usage: 862.2+ KB

== Info df2 (SalesFINAL12312016) ==
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12825363 entries, 0 to 12825362
Data columns (total 14 co